In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [43]:
##Load the dataset
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [44]:
#preprocessing the data
##Dropping unnecessary columns
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [45]:
##Encoding categorical variables
label_encoder_gender = LabelEncoder()
print(df['Gender'].unique())
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])
label_encoder_gender.classes_


['Female' 'Male']


array(['Female', 'Male'], dtype=object)

In [46]:
#One-hot encoding Geography
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
geo_encoded = ohe.fit_transform(df[['Geography']])
geo_encoded

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [47]:
ohe.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [48]:
# Converting the sparse matrix to a DataFrame
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(), columns=ohe.get_feature_names_out(['Geography']))
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [49]:
#Concatenating the encoded Geography columns with the original DataFrame
df = pd.concat([df, geo_encoded_df], axis=1)
df.drop('Geography', axis=1, inplace=True)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [50]:
##Saving Encoders
with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)
with open('ohe_geography.pkl', 'wb') as f:
    pickle.dump(ohe, f)


In [51]:
#Diving the dataset into features and target variable
X = df.drop('Exited', axis=1)
y = df['Exited']


In [52]:
##Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Scaling the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [53]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [54]:
##saving the scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [55]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


ANN IMPLEMENTATION


In [56]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [57]:
##Building the ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)) ,
    Dropout(0.5), ##HL1
    Dense(32, activation='relu'),
    Dropout(0.5), ##HL2
    Dense(1, activation='sigmoid') # Output layer for binary classification
])

/Users/mariebiskoot/Coding/Ann_Imp/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [58]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [59]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

In [60]:
#Compiling the model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [61]:
##Setup the Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


In [62]:
##Setup Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [63]:
#Training the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100,
                    callbacks=[tensorboard_callback, early_stopping])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 713us/step - accuracy: 0.7673 - loss: 0.5232 - val_accuracy: 0.8420 - val_loss: 0.3947
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step - accuracy: 0.8316 - loss: 0.4092 - val_accuracy: 0.8360 - val_loss: 0.3703
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 491us/step - accuracy: 0.8308 - loss: 0.3959 - val_accuracy: 0.8520 - val_loss: 0.3531
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 482us/step - accuracy: 0.8344 - loss: 0.3943 - val_accuracy: 0.8390 - val_loss: 0.3673
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 483us/step - accuracy: 0.8338 - loss: 0.3933 - val_accuracy: 0.8530 - val_loss: 0.3534
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 474us/step - accuracy: 0.8403 - loss: 0.3893 - val_accuracy: 0.8475 - val_loss: 0.3636
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 468us/step - accuracy: 0.8436 - loss: 0.3797 - val_accuracy: 0.8495 - val_loss: 0.3580
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 467us/step - accuracy: 0.8515 - loss: 0

In [64]:
#Saving the model
model.save('model.h5')

In [65]:
#Load Tensorboard
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [66]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 20962), started 0:34:15 ago. (Use '!kill 20962' to kill it.)